# Django Module Test - Solutions

This test covers all major topics from the Django module including:
- Django basics (project structure, apps, settings)
- Models and ORM (defining models, QuerySets, CRUD)
- Views and URLs (FBVs, CBVs, URL routing)
- Templates (DTL, inheritance, static files)
- Forms and Admin (form handling, ModelForms, admin customization)

## Instructions

1. Read each question carefully before answering
2. Write your answers in the provided code cells
3. For code questions, write syntactically correct Python/Django code
4. For conceptual questions, provide clear and concise explanations
5. Do not import actual Django modules - focus on demonstrating knowledge of syntax and concepts

---

## Part 1: Django Basics (Project Structure, Apps, Settings)

### Question 1: Project Structure

When you run `django-admin startproject mysite`, Django creates a specific directory structure.

**Task:** List the files that are created in the inner `mysite/` directory (not the outer one) and briefly describe the purpose of each file (2-3 words per file).

In [ ]:
# SOLUTION
"""
Files in the inner mysite/ directory:

1. __init__.py    - Python package marker
2. settings.py    - Project configuration settings
3. urls.py        - Root URL configuration
4. asgi.py        - ASGI application entry
5. wsgi.py        - WSGI application entry

Note: manage.py is in the OUTER directory, not the inner one.
"""

### Question 2: Settings Configuration

You are setting up a new Django project and need to configure the following in `settings.py`:
1. Add a new app called `blog` to the project
2. Configure the database to use PostgreSQL with:
   - Database name: `myblog_db`
   - User: `admin`
   - Password: `secret123`
   - Host: `localhost`
   - Port: `5432`

**Task:** Write the Python code for both configurations as they would appear in `settings.py`.

In [ ]:
# SOLUTION

# 1. Adding the blog app to INSTALLED_APPS
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    # Custom apps
    'blog',  # or 'blog.apps.BlogConfig' for explicit app config
]

# 2. PostgreSQL database configuration
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'myblog_db',
        'USER': 'admin',
        'PASSWORD': 'secret123',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}

### Question 3: Management Commands

**Task:** For each scenario below, write the exact Django management command you would use:

a) Create a new app called `products`

b) Create migration files after modifying a model

c) Apply pending migrations to the database

d) Create a superuser for the admin site

e) Start the development server on port 8080

In [ ]:
# SOLUTION

# a) Create a new app called products
# python manage.py startapp products

# b) Create migration files after modifying a model
# python manage.py makemigrations
# Or for a specific app:
# python manage.py makemigrations products

# c) Apply pending migrations to the database
# python manage.py migrate

# d) Create a superuser for the admin site
# python manage.py createsuperuser

# e) Start the development server on port 8080
# python manage.py runserver 8080
# Or with explicit host:
# python manage.py runserver 0.0.0.0:8080

---

## Part 2: Models and ORM

### Question 4: Model Definition

You are building a library management system. Create a Django model called `Book` with the following requirements:

- `title`: A string field, max 200 characters, required
- `author`: A string field, max 100 characters, required
- `isbn`: A string field, exactly 13 characters, must be unique
- `publication_date`: A date field, optional
- `price`: A decimal field with max 6 digits and 2 decimal places
- `is_available`: A boolean field, defaults to True
- `created_at`: A datetime field that auto-sets when created

Also add:
- A `__str__` method that returns the title and author
- A Meta class that orders books by title

In [ ]:
# SOLUTION
from django.db import models


class Book(models.Model):
    """Model representing a book in the library."""
    
    title = models.CharField(max_length=200)
    author = models.CharField(max_length=100)
    isbn = models.CharField(max_length=13, unique=True)
    publication_date = models.DateField(null=True, blank=True)
    price = models.DecimalField(max_digits=6, decimal_places=2)
    is_available = models.BooleanField(default=True)
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        ordering = ['title']
    
    def __str__(self):
        return f"{self.title} by {self.author}"

### Question 5: Model Relationships

Extend the library system with two more models:

1. `Category` model with:
   - `name`: CharField, max 50 characters
   - Books can belong to multiple categories, and categories can have multiple books

2. `Review` model with:
   - `book`: ForeignKey to Book (if book is deleted, delete reviews)
   - `reviewer_name`: CharField, max 100 characters
   - `rating`: IntegerField (1-5)
   - `comment`: TextField
   - `created_at`: auto-set datetime

**Task:** Write both model classes with proper relationships.

In [ ]:
# SOLUTION
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator


class Category(models.Model):
    """Model representing a book category."""
    
    name = models.CharField(max_length=50)
    # ManyToMany relationship - can be defined on either model
    books = models.ManyToManyField('Book', related_name='categories', blank=True)
    
    class Meta:
        verbose_name_plural = 'categories'
    
    def __str__(self):
        return self.name


class Review(models.Model):
    """Model representing a book review."""
    
    book = models.ForeignKey(
        'Book',
        on_delete=models.CASCADE,
        related_name='reviews'
    )
    reviewer_name = models.CharField(max_length=100)
    rating = models.IntegerField(
        validators=[MinValueValidator(1), MaxValueValidator(5)]
    )
    comment = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    
    def __str__(self):
        return f"Review for {self.book.title} by {self.reviewer_name}"


# Alternative: ManyToMany defined on Book model instead
# class Book(models.Model):
#     ...
#     categories = models.ManyToManyField(Category, related_name='books', blank=True)

### Question 6: QuerySet Operations

Using the `Book` model from Question 4, write QuerySet operations for each scenario:

a) Get all available books ordered by price (ascending)

b) Get books published after January 1, 2020

c) Get books where the title contains "Python" (case-insensitive)

d) Get the count of books by author "John Smith"

e) Get books priced between $10 and $50, only returning title and price fields

f) Get the first 5 most expensive books

g) Update all books by "Old Author" to change author to "New Author"

h) Delete all books that are not available

In [ ]:
# SOLUTION
from datetime import date

# a) Get all available books ordered by price (ascending)
available_books = Book.objects.filter(is_available=True).order_by('price')

# b) Get books published after January 1, 2020
recent_books = Book.objects.filter(publication_date__gt=date(2020, 1, 1))
# Alternative: publication_date__gte for "on or after"

# c) Get books where the title contains "Python" (case-insensitive)
python_books = Book.objects.filter(title__icontains='Python')

# d) Get the count of books by author "John Smith"
john_smith_count = Book.objects.filter(author='John Smith').count()

# e) Get books priced between $10 and $50, only returning title and price fields
mid_priced_books = Book.objects.filter(
    price__gte=10,
    price__lte=50
).values('title', 'price')
# Alternative using range:
# Book.objects.filter(price__range=(10, 50)).values('title', 'price')

# f) Get the first 5 most expensive books
expensive_books = Book.objects.order_by('-price')[:5]

# g) Update all books by "Old Author" to change author to "New Author"
Book.objects.filter(author='Old Author').update(author='New Author')

# h) Delete all books that are not available
Book.objects.filter(is_available=False).delete()

---

## Part 3: Views and URLs

### Question 7: Function-Based Views

Write a function-based view called `book_list` that:
1. Handles both GET and POST requests
2. On GET: retrieves all available books and renders `books/book_list.html` with the books in context
3. On POST: expects `title` in POST data, filters books by title (contains), and renders the same template

Include all necessary imports.

In [ ]:
# SOLUTION
from django.shortcuts import render
from django.http import HttpRequest, HttpResponse
from .models import Book


def book_list(request: HttpRequest) -> HttpResponse:
    """Display a list of books, with optional title search.
    
    Args:
        request: The HTTP request object.
        
    Returns:
        Rendered book list template with books in context.
    """
    if request.method == 'POST':
        # Get the search title from POST data
        title = request.POST.get('title', '')
        # Filter books by title (case-insensitive contains)
        books = Book.objects.filter(
            title__icontains=title,
            is_available=True
        )
    else:
        # GET request - show all available books
        books = Book.objects.filter(is_available=True)
    
    context = {
        'books': books,
    }
    return render(request, 'books/book_list.html', context)

### Question 8: Class-Based Views

Rewrite the following functionality using Django's generic class-based views:

1. A `BookListView` that displays all books (paginated, 10 per page)
2. A `BookDetailView` that shows a single book by its primary key
3. A `BookCreateView` that provides a form to create a new book (redirect to list on success)
4. A `BookDeleteView` that confirms and deletes a book (redirect to list on success)

Include all necessary imports.

In [ ]:
# SOLUTION
from django.views.generic import ListView, DetailView, CreateView, DeleteView
from django.urls import reverse_lazy
from .models import Book


class BookListView(ListView):
    """Display a paginated list of all books."""
    
    model = Book
    template_name = 'books/book_list.html'
    context_object_name = 'books'
    paginate_by = 10


class BookDetailView(DetailView):
    """Display details of a single book."""
    
    model = Book
    template_name = 'books/book_detail.html'
    context_object_name = 'book'
    # pk_url_kwarg = 'pk'  # Default, can be customized


class BookCreateView(CreateView):
    """Form view to create a new book."""
    
    model = Book
    template_name = 'books/book_form.html'
    fields = ['title', 'author', 'isbn', 'publication_date', 'price', 'is_available']
    success_url = reverse_lazy('books:book-list')


class BookDeleteView(DeleteView):
    """Confirmation view to delete a book."""
    
    model = Book
    template_name = 'books/book_confirm_delete.html'
    context_object_name = 'book'
    success_url = reverse_lazy('books:book-list')

### Question 9: URL Configuration

Write the URL configuration (`urls.py`) for a `books` app that includes:

1. `/books/` -> `book_list` view (name: `book-list`)
2. `/books/<id>/` -> `book_detail` view (name: `book-detail`) where id is an integer
3. `/books/create/` -> `BookCreateView` (name: `book-create`)
4. `/books/<id>/delete/` -> `BookDeleteView` (name: `book-delete`)
5. `/books/category/<slug>/` -> `category_books` view (name: `category-books`) where slug is a slug

Include the app_name for namespacing.

In [ ]:
# SOLUTION
# books/urls.py

from django.urls import path
from . import views
from .views import BookCreateView, BookDeleteView

app_name = 'books'

urlpatterns = [
    # Book list view
    path('', views.book_list, name='book-list'),
    
    # Book detail view - <int:id> captures an integer parameter
    path('<int:id>/', views.book_detail, name='book-detail'),
    
    # Book create view (CBV)
    path('create/', BookCreateView.as_view(), name='book-create'),
    
    # Book delete view (CBV)
    path('<int:id>/delete/', BookDeleteView.as_view(), name='book-delete'),
    
    # Category books view - <slug:slug> captures a slug parameter
    path('category/<slug:slug>/', views.category_books, name='category-books'),
]

# Note: In the main project urls.py, you would include this:
# from django.urls import path, include
# urlpatterns = [
#     path('books/', include('books.urls')),
# ]

---

## Part 4: Templates (DTL, Inheritance, Static Files)

### Question 10: Template Inheritance

Create a base template (`base.html`) that:
1. Has a proper HTML5 structure
2. Loads static files
3. Includes a CSS file `css/styles.css`
4. Has three blocks: `title`, `content`, and `extra_js`
5. Has a basic navigation with links using the `url` template tag for:
   - Home (`home`)
   - Book List (`books:book-list`)

Then create a child template (`book_list.html`) that:
1. Extends the base template
2. Sets the title to "Book List"
3. Displays books in a loop with title, author, and price
4. Shows "No books available" if the list is empty
5. Each book title links to its detail page using the `url` tag

In [ ]:
# SOLUTION - base.html
base_html = """
{% load static %}
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}My Library{% endblock %}</title>
    <link rel="stylesheet" href="{% static 'css/styles.css' %}">
</head>
<body>
    <nav>
        <ul>
            <li><a href="{% url 'home' %}">Home</a></li>
            <li><a href="{% url 'books:book-list' %}">Book List</a></li>
        </ul>
    </nav>
    
    <main>
        {% block content %}
        {% endblock %}
    </main>
    
    {% block extra_js %}
    {% endblock %}
</body>
</html>
"""
print(base_html)

In [ ]:
# SOLUTION - book_list.html
book_list_html = """
{% extends 'base.html' %}

{% block title %}Book List{% endblock %}

{% block content %}
<h1>Book List</h1>

{% if books %}
    <ul>
    {% for book in books %}
        <li>
            <a href="{% url 'books:book-detail' book.id %}">{{ book.title }}</a>
            by {{ book.author }} - ${{ book.price }}
        </li>
    {% endfor %}
    </ul>
{% else %}
    <p>No books available.</p>
{% endif %}
{% endblock %}
"""
print(book_list_html)

### Question 11: Template Tags and Filters

Given a context with:
```python
context = {
    'book': {'title': 'python basics', 'price': 29.99, 'published': datetime(2023, 5, 15)},
    'description': '<p>A great book about <strong>Python</strong></p>',
    'tags': ['programming', 'python', 'beginner'],
}
```

Write the Django template code to:

a) Display the title in title case ("Python Basics")

b) Display the price formatted as currency with 2 decimal places

c) Display the published date in format "May 15, 2023"

d) Display the description with HTML tags rendered (not escaped)

e) Display tags as a comma-separated string

f) Display "New!" if the book was published less than 1 year ago, otherwise "Classic"

In [ ]:
# SOLUTION
template_code = """
{# a) Display the title in title case ("Python Basics") #}
{{ book.title|title }}

{# b) Display the price formatted as currency with 2 decimal places #}
${{ book.price|floatformat:2 }}
{# Alternative: using stringformat #}
${{ book.price|stringformat:".2f" }}

{# c) Display the published date in format "May 15, 2023" #}
{{ book.published|date:"F j, Y" }}

{# d) Display the description with HTML tags rendered (not escaped) #}
{{ description|safe }}
{# Alternative: using autoescape off #}
{% autoescape off %}{{ description }}{% endautoescape %}

{# e) Display tags as a comma-separated string #}
{{ tags|join:", " }}

{# f) Display "New!" if the book was published less than 1 year ago, otherwise "Classic" #}
{% if book.published|timesince < "365 days" %}
    New!
{% else %}
    Classic
{% endif %}

{# More reliable approach using a custom filter or view logic: #}
{# In the view, calculate and pass is_new boolean, then: #}
{% if is_new %}New!{% else %}Classic{% endif %}
"""
print(template_code)

---

## Part 5: Forms and Admin

### Question 12: Django Forms

Create a `BookSearchForm` using Django's forms module that has:
1. `title` - optional CharField with max_length 200 and placeholder "Search by title"
2. `min_price` - optional DecimalField with min_value 0
3. `max_price` - optional DecimalField with min_value 0
4. `is_available` - optional BooleanField, not required

Add a `clean` method that validates max_price is greater than min_price when both are provided.

In [ ]:
# SOLUTION
from django import forms
from decimal import Decimal


class BookSearchForm(forms.Form):
    """Form for searching books with various filters."""
    
    title = forms.CharField(
        max_length=200,
        required=False,
        widget=forms.TextInput(attrs={'placeholder': 'Search by title'})
    )
    min_price = forms.DecimalField(
        min_value=Decimal('0'),
        required=False,
        decimal_places=2
    )
    max_price = forms.DecimalField(
        min_value=Decimal('0'),
        required=False,
        decimal_places=2
    )
    is_available = forms.BooleanField(required=False)
    
    def clean(self):
        """Validate that max_price is greater than min_price.
        
        Returns:
            The cleaned data dictionary.
            
        Raises:
            ValidationError: If max_price is less than or equal to min_price.
        """
        cleaned_data = super().clean()
        min_price = cleaned_data.get('min_price')
        max_price = cleaned_data.get('max_price')
        
        if min_price is not None and max_price is not None:
            if max_price <= min_price:
                raise forms.ValidationError(
                    'Maximum price must be greater than minimum price.'
                )
        
        return cleaned_data

### Question 13: ModelForm

Create a `BookForm` ModelForm for the `Book` model (from Question 4) that:
1. Includes all fields except `created_at`
2. Uses a `Textarea` widget for any description field
3. Uses a `DateInput` widget with type='date' for `publication_date`
4. Adds a CSS class 'form-control' to all fields
5. Customizes labels for `isbn` to "ISBN Number" and `is_available` to "Currently Available"

In [ ]:
# SOLUTION
from django import forms
from .models import Book


class BookForm(forms.ModelForm):
    """ModelForm for creating and editing Book instances."""
    
    class Meta:
        model = Book
        exclude = ['created_at']
        # Alternative: fields = ['title', 'author', 'isbn', 'publication_date', 'price', 'is_available']
        
        widgets = {
            'publication_date': forms.DateInput(attrs={'type': 'date'}),
            # If there was a description field:
            # 'description': forms.Textarea(attrs={'rows': 4}),
        }
        
        labels = {
            'isbn': 'ISBN Number',
            'is_available': 'Currently Available',
        }
    
    def __init__(self, *args, **kwargs):
        """Initialize form and add CSS class to all fields."""
        super().__init__(*args, **kwargs)
        
        # Add 'form-control' class to all fields
        for field_name, field in self.fields.items():
            existing_classes = field.widget.attrs.get('class', '')
            field.widget.attrs['class'] = f"{existing_classes} form-control".strip()

### Question 14: Admin Configuration

Write an admin configuration for the `Book` model that:
1. Displays `title`, `author`, `isbn`, `price`, and `is_available` in the list view
2. Adds filters for `is_available` and `publication_date`
3. Adds search functionality for `title`, `author`, and `isbn`
4. Makes `isbn` read-only in the edit form
5. Orders the list by `-created_at` (newest first)
6. Adds a custom admin action to mark selected books as available

Include the model registration.

In [ ]:
# SOLUTION
from django.contrib import admin
from .models import Book


@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    """Admin configuration for the Book model."""
    
    # List view configuration
    list_display = ['title', 'author', 'isbn', 'price', 'is_available']
    list_filter = ['is_available', 'publication_date']
    search_fields = ['title', 'author', 'isbn']
    ordering = ['-created_at']
    
    # Edit form configuration
    readonly_fields = ['isbn']
    
    # Custom admin actions
    actions = ['mark_as_available']
    
    @admin.action(description='Mark selected books as available')
    def mark_as_available(self, request, queryset):
        """Mark selected books as available.
        
        Args:
            request: The HTTP request object.
            queryset: QuerySet of selected Book objects.
        """
        updated_count = queryset.update(is_available=True)
        self.message_user(
            request,
            f'{updated_count} book(s) marked as available.'
        )


# Alternative registration without decorator:
# admin.site.register(Book, BookAdmin)

### Question 15: Putting It All Together

You are building a view that handles a book creation form. Write a function-based view called `create_book` that:

1. Uses the `BookForm` ModelForm
2. On GET: displays an empty form
3. On POST:
   - Validates the form
   - If valid: saves the book, adds a success message, and redirects to `books:book-detail`
   - If invalid: re-renders the form with errors
4. Uses Django's messages framework
5. Requires the user to be logged in (use decorator)

Include all necessary imports.

In [ ]:
# SOLUTION
from django.shortcuts import render, redirect
from django.http import HttpRequest, HttpResponse
from django.contrib import messages
from django.contrib.auth.decorators import login_required
from .forms import BookForm


@login_required
def create_book(request: HttpRequest) -> HttpResponse:
    """Handle book creation form.
    
    On GET, displays an empty book creation form.
    On POST, validates and saves the form, then redirects to detail view.
    
    Args:
        request: The HTTP request object.
        
    Returns:
        HttpResponse with the rendered form or redirect to book detail.
    """
    if request.method == 'POST':
        form = BookForm(request.POST)
        
        if form.is_valid():
            # Save the book and get the instance
            book = form.save()
            
            # Add success message
            messages.success(
                request,
                f'Book "{book.title}" was created successfully!'
            )
            
            # Redirect to the book detail page
            return redirect('books:book-detail', id=book.id)
        # If form is invalid, fall through to render with errors
    else:
        # GET request - display empty form
        form = BookForm()
    
    context = {
        'form': form,
    }
    return render(request, 'books/book_form.html', context)

---

## End of Test

Review your answers before submission. Make sure all code is syntactically correct and follows Django best practices.